#### Test accuracy: 0.7554

In [50]:
import keras

from sklearn.cross_validation import train_test_split
import pandas as pd
from keras.wrappers.scikit_learn import KerasClassifier, KerasRegressor
from sklearn.model_selection import GridSearchCV
from keras import backend as K
from keras.layers import Dense, Dropout, Flatten, Activation
import numpy as np
import matplotlib.pyplot as plt
from keras.preprocessing import image
from scipy.misc import imread
import scipy.io as sio
import re
from keras.applications import VGG16
from keras.applications.vgg16 import preprocess_input
from sklearn.linear_model import LogisticRegressionCV
from keras.layers import Dense, Dropout, Flatten, Activation
from keras.models import Sequential
import os

In [51]:
path_to_pets = "./pets/"

In [39]:
# build the VGG16 network
# pip install h5py
model = VGG16(include_top=False,
                           weights='imagenet')

cwd = os.getcwd()
parent_path = os.path.split(os.getcwd())[0]
print ("parent path is {}".format(parent_path))


# from keras.preprocessing import image
dir_list = os.listdir(path_to_pets)
dir_list = [f for f in os.listdir(path_to_pets) if re.match(r'[^\\]+\.jpg', f)]

print ("there are {} images in pets directory".format(len(dir_list)))


images_list = [image.load_img(path_to_pets + f, target_size=(224, 224))
                 for f in dir_list]

print(len(images_list))

X = np.array([image.img_to_array(img) for img in images_list])

print ("X.shape: {}".format(X.shape))

# from keras.applications.vgg16 import preprocess_input
X_pre = preprocess_input(X)
features = model.predict(X_pre)

print("features.shape: {}".format(features.shape))
#np.savetxt('features.out', features)   # 

features_ = features.reshape(7390, -1)
print("features_.shape: {}".format(features_.shape))
np.savetxt('features_.out', features_)   # 

# create labels e.g. basset_hound

labels = []
for f in dir_list:
    jpg_removed = f.split(".")[0]
    labels.append(jpg_removed.rsplit("_",1)[0])


/home/yh2901/.local/lib/python3.5/site-packages/sklearn/grid_search.py:43: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


parent path is /home/yh2901
there are 7390 images in pets directory
7390
X.shape: (7390, 224, 224, 3)
features.shape: (7390, 7, 7, 512)
features_.shape: (7390, 25088)


In [3]:
np.savetxt('features_.out', features_) 

In [4]:
ls -l

total 5310084
-rw-rw-r-- 1 yh2901 yh2901 4635008000 Apr 29 15:11 features_.out
-rw-r--r-- 1 yh2901 yh2901  802191360 Apr 29 05:13 images.tar
drwxr-xr-x 2 yh2901 yh2901     303104 Jun 18  2012 pets/
-rw-rw-r-- 1 yh2901 yh2901       6067 Apr 29 15:11 task4-notebook.ipynb
-rw-rw-r-- 1 yh2901 yh2901       2734 Apr 29 14:17 test.py


In [6]:
from sklearn import preprocessing
features_scaled = preprocessing.scale(features_)

/home/yh2901/.local/lib/python3.5/site-packages/sklearn/preprocessing/data.py:160: UserWarning: Numerical issues were encountered when centering the data and might not be solved. Dataset may contain too large values. You may need to prescale your features.
  warnings.warn("Numerical issues were encountered "
/home/yh2901/.local/lib/python3.5/site-packages/sklearn/preprocessing/data.py:177: UserWarning: Numerical issues were encountered when scaling the data and might not be solved. The standard deviation of the data is probably very close to 0. 
  warnings.warn("Numerical issues were encountered "


In [18]:
features_scaled.shape

(7390, 25088)

In [20]:
labels[:3]

['scottish_terrier', 'beagle', 'wheaten_terrier']

In [25]:
sets = list(set(labels))
from collections import defaultdict
label_map_int = defaultdict(int)
for i in range(37):
    label_map_int[sets[i]] = i

In [26]:
label_map_int

defaultdict(int,
            {'Abyssinian': 14,
             'Bengal': 11,
             'Birman': 1,
             'Bombay': 23,
             'British_Shorthair': 4,
             'Egyptian_Mau': 15,
             'Maine_Coon': 6,
             'Persian': 28,
             'Ragdoll': 21,
             'Russian_Blue': 32,
             'Siamese': 17,
             'Sphynx': 9,
             'american_bulldog': 30,
             'american_pit_bull_terrier': 24,
             'basset_hound': 18,
             'beagle': 2,
             'boxer': 20,
             'chihuahua': 16,
             'english_cocker_spaniel': 36,
             'english_setter': 22,
             'german_shorthaired': 31,
             'great_pyrenees': 8,
             'havanese': 34,
             'japanese_chin': 0,
             'keeshond': 29,
             'leonberger': 7,
             'miniature_pinscher': 19,
             'newfoundland': 35,
             'pomeranian': 27,
             'pug': 3,
             'saint_bernard': 26,

In [27]:
labels_ = [label_map_int[l] for l in labels]

In [29]:
dir_list[0]

'scottish_terrier_190.jpg'

In [30]:
labels_dummy = keras.utils.to_categorical(labels_)

In [31]:
X_train, X_test, y_train, y_test = train_test_split(features_scaled, labels_dummy, stratify=labels)

print (len(X_train), len(y_train), len(X_test), len(y_test))
print(X_train[0].shape)

5542 5542 1848 1848
(25088,)


#### model selection

In [37]:
def make_nnet(hidden_size=16):
    r"""
    Returns
    -------
    model : model itself
    """
    
    num_class = 37
    input_shape = (25088,)
    
    model = Sequential()
    model.add(Dense(hidden_size, input_shape=input_shape, activation="relu"))
    #model.add(Dense(hidden_size/2, activation="relu"))
    model.add(Dense(num_class, activation="softmax"))
    
    model.compile(loss="categorical_crossentropy", 
                  optimizer="adam", 
                  metrics=['accuracy'])

    return  model

In [40]:
clf = KerasClassifier(make_nnet)
param_grid = {'epochs': [16,32], 'hidden_size': [16,32], 'batch_size':[128]}
grid = GridSearchCV(clf, param_grid=param_grid, cv=5)
grid.fit(X_train, y_train)

Epoch 1/16
4433/4433 [==============================] - 1s - loss: 2.7061 - acc: 0.2732     
Epoch 2/16
4433/4433 [==============================] - 1s - loss: 0.8145 - acc: 0.7839     
Epoch 3/16
4433/4433 [==============================] - 0s - loss: 0.3331 - acc: 0.9393     
Epoch 4/16
4433/4433 [==============================] - 0s - loss: 0.1577 - acc: 0.9761     
Epoch 5/16
4433/4433 [==============================] - 0s - loss: 0.0881 - acc: 0.9905     
Epoch 6/16
4433/4433 [==============================] - 0s - loss: 0.0594 - acc: 0.9932     
Epoch 7/16
4433/4433 [==============================] - 0s - loss: 0.0413 - acc: 0.9950     
Epoch 8/16
4433/4433 [==============================] - 0s - loss: 0.0317 - acc: 0.9950     
Epoch 9/16
4433/4433 [==============================] - 0s - loss: 0.0260 - acc: 0.9962     
Epoch 10/16
4433/4433 [==============================] - 0s - loss: 0.0220 - acc: 0.9968     
Epoch 11/16
4433/4433 [==============================] - 0s - loss: 0

4433/4433 [==============================] - 1s - loss: 0.0109 - acc: 0.9993     
Epoch 6/16
4433/4433 [==============================] - 1s - loss: 0.0081 - acc: 0.9995     
Epoch 7/16
4433/4433 [==============================] - 1s - loss: 0.0064 - acc: 0.9995     
Epoch 8/16
4433/4433 [==============================] - 1s - loss: 0.0054 - acc: 0.9995     
Epoch 9/16
4433/4433 [==============================] - 1s - loss: 0.0043 - acc: 0.9998     
Epoch 10/16
4433/4433 [==============================] - 1s - loss: 0.0035 - acc: 0.9998     
Epoch 11/16
4433/4433 [==============================] - 1s - loss: 0.0029 - acc: 1.0000     
Epoch 12/16
4433/4433 [==============================] - 1s - loss: 0.0024 - acc: 1.0000     
Epoch 13/16
4433/4433 [==============================] - 1s - loss: 0.0022 - acc: 1.0000     
Epoch 14/16
4433/4433 [==============================] - 1s - loss: 0.0019 - acc: 1.0000     
Epoch 15/16
4433/4433 [==============================] - 1s - loss: 0.0017 -

4433/4433 [==============================] - 0s - loss: 0.0224 - acc: 0.9973     
Epoch 10/32
4433/4433 [==============================] - 0s - loss: 0.0187 - acc: 0.9982     
Epoch 11/32
4433/4433 [==============================] - 0s - loss: 0.0166 - acc: 0.9982     
Epoch 12/32
4433/4433 [==============================] - 0s - loss: 0.0136 - acc: 0.9986     
Epoch 13/32
4433/4433 [==============================] - 0s - loss: 0.0117 - acc: 0.9984     
Epoch 14/32
4433/4433 [==============================] - 0s - loss: 0.0103 - acc: 0.9986     
Epoch 15/32
4433/4433 [==============================] - 1s - loss: 0.0089 - acc: 0.9991     
Epoch 16/32
4433/4433 [==============================] - 0s - loss: 0.0080 - acc: 0.9995     
Epoch 17/32
4433/4433 [==============================] - 0s - loss: 0.0070 - acc: 0.9993     
Epoch 18/32
4433/4433 [==============================] - 0s - loss: 0.0063 - acc: 0.9995     
Epoch 19/32
4433/4433 [==============================] - 0s - loss: 0.00

4434/4434 [==============================] - 0s - loss: 0.0027 - acc: 0.9998     
Epoch 32/32
4434/4434 [==============================] - 1s - loss: 2.6595 - acc: 0.2912     
Epoch 2/32
4434/4434 [==============================] - 0s - loss: 0.7350 - acc: 0.8124     
Epoch 3/32
4434/4434 [==============================] - 0s - loss: 0.2769 - acc: 0.9484     
Epoch 4/32
4434/4434 [==============================] - 0s - loss: 0.1185 - acc: 0.9905     
Epoch 5/32
4434/4434 [==============================] - 0s - loss: 0.0628 - acc: 0.9962     
Epoch 6/32
4434/4434 [==============================] - 0s - loss: 0.0414 - acc: 0.9966     
Epoch 7/32
4434/4434 [==============================] - 1s - loss: 0.0296 - acc: 0.9984     
Epoch 8/32
4434/4434 [==============================] - 0s - loss: 0.0229 - acc: 0.9986     
Epoch 9/32
4434/4434 [==============================] - 0s - loss: 0.0192 - acc: 0.9989     
Epoch 10/32
4434/4434 [==============================] - 0s - loss: 0.0165 - acc

4433/4433 [==============================] - 1s - loss: 9.9155e-04 - acc: 1.0000     
Epoch 21/32
4433/4433 [==============================] - 1s - loss: 9.1858e-04 - acc: 1.0000     
Epoch 22/32
4433/4433 [==============================] - 1s - loss: 8.5316e-04 - acc: 1.0000     
Epoch 23/32
4433/4433 [==============================] - 1s - loss: 7.9505e-04 - acc: 1.0000     
Epoch 24/32
4433/4433 [==============================] - 1s - loss: 7.4226e-04 - acc: 1.0000     
Epoch 25/32
4433/4433 [==============================] - 1s - loss: 6.9488e-04 - acc: 1.0000     
Epoch 26/32
4433/4433 [==============================] - 1s - loss: 6.5195e-04 - acc: 1.0000     
Epoch 27/32
4433/4433 [==============================] - 1s - loss: 6.1289e-04 - acc: 1.0000     
Epoch 28/32
4433/4433 [==============================] - 1s - loss: 5.7731e-04 - acc: 1.0000     
Epoch 29/32
4433/4433 [==============================] - 1s - loss: 5.4451e-04 - acc: 1.0000     
Epoch 30/32
4433/4433 [=========

4434/4434 [==============================] - 1s - loss: 0.0050 - acc: 1.0000     
Epoch 8/32
4434/4434 [==============================] - 1s - loss: 0.0041 - acc: 1.0000     
Epoch 9/32
4434/4434 [==============================] - 1s - loss: 0.0035 - acc: 1.0000     
Epoch 10/32
4434/4434 [==============================] - 1s - loss: 0.0030 - acc: 1.0000     - ETA: 0s - loss: 0.0029 - ac
Epoch 11/32
4434/4434 [==============================] - 1s - loss: 0.0026 - acc: 1.0000     
Epoch 12/32
4434/4434 [==============================] - 1s - loss: 0.0023 - acc: 1.0000     
Epoch 13/32
4434/4434 [==============================] - 1s - loss: 0.0020 - acc: 1.0000     
Epoch 14/32
4434/4434 [==============================] - 1s - loss: 0.0018 - acc: 1.0000     
Epoch 15/32
4434/4434 [==============================] - 1s - loss: 0.0016 - acc: 1.0000     - ETA: 0s - loss: 0.0017 - acc: 1.0
Epoch 16/32
4434/4434 [==============================] - 1s - loss: 0.0015 - acc: 1.0000     
Epoch 17/3

5542/5542 [==============================] - 1s - loss: 5.5270e-04 - acc: 1.0000     
Epoch 27/32
5542/5542 [==============================] - 1s - loss: 5.1751e-04 - acc: 1.0000     
Epoch 28/32
5542/5542 [==============================] - 1s - loss: 4.8589e-04 - acc: 1.0000     
Epoch 29/32
5542/5542 [==============================] - 1s - loss: 4.5679e-04 - acc: 1.0000     
Epoch 30/32
5542/5542 [==============================] - 1s - loss: 4.3008e-04 - acc: 1.0000     
Epoch 31/32
5542/5542 [==============================] - 1s - loss: 4.0582e-04 - acc: 1.0000     
Epoch 32/32
5542/5542 [==============================] - 1s - loss: 3.8333e-04 - acc: 1.0000     


GridSearchCV(cv=5, error_score='raise',
       estimator=<keras.wrappers.scikit_learn.KerasClassifier object at 0x7f140d1d4b38>,
       fit_params={}, iid=True, n_jobs=1,
       param_grid={'hidden_size': [16, 32], 'epochs': [16, 32], 'batch_size': [128]},
       pre_dispatch='2*n_jobs', refit=True, scoring=None, verbose=0)

In [47]:
print ("best params: {}".format(grid.best_params_))
print ("scores: {} ".format(grid.grid_scores_))

best params: {'batch_size': 128, 'epochs': 32, 'hidden_size': 32}
scores: [mean: 0.64706, std: 0.00701, params: {'batch_size': 128, 'epochs': 16, 'hidden_size': 16}, mean: 0.72681, std: 0.01036, params: {'batch_size': 128, 'epochs': 16, 'hidden_size': 32}, mean: 0.67322, std: 0.01239, params: {'batch_size': 128, 'epochs': 32, 'hidden_size': 16}, mean: 0.73241, std: 0.00732, params: {'batch_size': 128, 'epochs': 32, 'hidden_size': 32}] 


In [46]:
grid.grid_scores_

[mean: 0.64706, std: 0.00701, params: {'batch_size': 128, 'epochs': 16, 'hidden_size': 16},
 mean: 0.72681, std: 0.01036, params: {'batch_size': 128, 'epochs': 16, 'hidden_size': 32},
 mean: 0.67322, std: 0.01239, params: {'batch_size': 128, 'epochs': 32, 'hidden_size': 16},
 mean: 0.73241, std: 0.00732, params: {'batch_size': 128, 'epochs': 32, 'hidden_size': 32}]

The best parameters are 128 batch size, 32 epoches, and 32 hidden layer, which achieves 0.7341 on the training data

In [48]:
def train_nnet_shallow( x_train, x_test, y_train, y_test):
    r"""
    Returns
    -------
    hisotry_callback : <class 'keras.callbacks.History'>
        used for retrospective examiniation
    """
    
    num_class = 37
    epochs = 32
    batch_size = 128
    input_shape = (25088, ) 
    model = Sequential()
    model.add(Dense(32, input_shape=input_shape, activation="relu"))
    #model.add(Dense(16, activation="relu"))
    model.add(Dense(num_class, activation="softmax"))
    
    model.compile(loss="categorical_crossentropy", 
                  optimizer="adam", 
                  metrics=['accuracy'])
    
    history_callback = model.fit(
                  x_train, 
                  y_train, 
                  verbose=1, 
                  batch_size=batch_size,
                  validation_data=(x_test, y_test),
                  epochs=epochs
              )
    
    score = model.evaluate(x_test, y_test, verbose=0)
    
    print("Test loss: {}".format(score[0]))
    print("Test accuracy: {}".format(score[1]))
    #print(model.summary())

    return history_callback

%time callback = train_nnet_shallow(X_train, X_test, y_train, y_test)

Train on 5542 samples, validate on 1848 samples
Epoch 1/32
5542/5542 [==============================] - 2s - loss: 2.2246 - acc: 0.4332 - val_loss: 1.1894 - val_acc: 0.6726
Epoch 2/32
5542/5542 [==============================] - 1s - loss: 0.1870 - acc: 0.9551 - val_loss: 0.9641 - val_acc: 0.7170
Epoch 3/32
5542/5542 [==============================] - 1s - loss: 0.0272 - acc: 0.9986 - val_loss: 0.9240 - val_acc: 0.7240
Epoch 4/32
5542/5542 [==============================] - 1s - loss: 0.0115 - acc: 0.9998 - val_loss: 0.9058 - val_acc: 0.7289
Epoch 5/32
5542/5542 [==============================] - 1s - loss: 0.0079 - acc: 0.9998 - val_loss: 0.8980 - val_acc: 0.7316
Epoch 6/32
5542/5542 [==============================] - 1s - loss: 0.0060 - acc: 1.0000 - val_loss: 0.8925 - val_acc: 0.7343
Epoch 7/32
5542/5542 [==============================] - 1s - loss: 0.0048 - acc: 1.0000 - val_loss: 0.8873 - val_acc: 0.7359
Epoch 8/32
5542/5542 [==============================] - 1s - loss: 0.0039 - a